In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint,uniform

import time

# General Parameters

In [2]:
n_pop = 10000
n_loc = 2601
n_net = 200
n_overlap = 100

labels = [0,1,2,3,4] # Number of states a person can be in, [S,E,I,R,D] (Not very important, just for bookkeeping)

person_attr = 4  # 4 attributes for a person: state, home, network, current location

# Rates

transmit_rate = 0.2
e_to_i_rate = 0.2
recovery_rate = 0.1
death_fraction= 0.01
background_infection_rate = 0#0.01

random_test_rate =0.1
test_sensitivity =0.99
finish_test_rate =1.0

# Step 1: Create a population

Individuals are defined by `['state', 'home_location', 'network_location', 'current_location' ]`, for example: 

`[0, 0, 4, 0]` is an individual who is susceptible, whose home is `0` and whose work is `4`, and who is currently at home.

In [3]:

def create_person(state, home):                  # Function to create a person with certain attributes
    net = randint(0,n_net)                       # Assign a random network from 0 to n_net (here, 200)
    return np.array([state, home , net , home ]) # Create a person in state 'state', initially at home


pop = np.zeros((n_pop,person_attr),int)          # Empty population
link_weight = np.zeros(n_pop,float)              # Array for link weights
is_confined = np.zeros(n_pop,bool)               # Boolean array, is confined?
is_dead     = np.zeros(n_pop,bool)               # Boolean array, is dead?
being_tested = np.zeros(n_pop,bool)              # Boolean array, is being tested?

n_per_location = np.zeros((n_loc,#+n_net,          # Array to hold total number of individuals per state 
                           len(labels)),int)     # for all locations: both homes and networks

# Initial populations ###########

n_inf = 10
n_exp = 0
n_rec = 0
n_dead= 0
n_sus = n_pop - n_inf

output = np.array([0,n_sus, n_exp, n_inf, n_rec, n_dead],int)  # Output of the form t,S,E,I,R,D

#%
inf_by_loc = np.zeros(n_loc,int)
#%
##################################

for i in range(n_overlap, n_overlap+n_inf):      # Set 10 infected person  with random network
    pop[i] = create_person(2,i)

    n_per_location[ pop[i][3] ][2] +=  1         # Increment infected in each person's location
    inf_by_loc[pop[i][3]] +=1
    link_weight[i] = 0.1

for i in range(n_inf,n_pop):
    home = randint(n_overlap,n_loc)              # Assign random homes for remainder with random network
    pop[i] = create_person(0,home)

    n_per_location[  pop[i][3] ][0] +=  1        # Increment susceptibles in each person's location

    link_weight[i] = 0.1

print(np.sum(n_per_location))





10000


In [4]:
def Gillespie_Run(tf,output):
    
    n_sus = output[1]
    n_exp = output[2]
    n_inf = output[3]
    n_rec = output[4]
    n_dead= output[5]

    t = 0.0
    t_out = 0
    
    counter=0
    while(t<tf):#n_exp+n_inf>0):
            
            
        if(t>=t_out):
            output = np.vstack((output,[t_out,n_sus,n_exp,n_inf,n_rec,n_dead]))
            t_out += 1
            print(t)


        is_not_conf_or_dead = np.logical_not(is_confined+is_dead)   # Boolean array of people neither confined nor dead 

        susc_loc = pop[np.where(pop[:,0]==0)[0]][:,3]               # Locations of all susceptible individuals
        
    #     This complicated line below basically looks where susceptible individuals are, finds their corresponding
    #     current location, and then sums up the total number of infected people in all of those locations.
    #     The number will be larger than n_inf, since the same location may have multiple susceptible individuals!
        
#         a1 = np.sum(n_per_location[susc_loc][:,2]) * transmit_rate + (n_sus+n_inf)*background_infection_rate
#         print("fancy "+str(a1))
#         a1 = 0
#         for i in range(0,n_pop):
#             if pop[i][0]==0:  # if S
#                 a1 = a1 + transmit_rate * inf_by_loc[pop[i][3]] + background_infection_rate
#         print("slow "+str(a1))

        a1 = np.sum(inf_by_loc[susc_loc]) * transmit_rate + n_sus*background_infection_rate

        a2 = n_exp*e_to_i_rate
        a3 = n_inf*recovery_rate
        a4 = np.sum(is_not_conf_or_dead)*random_test_rate           # If the rates become person specific, move the product inside the sum
        a5 = np.sum(being_tested)*finish_test_rate                  # idem
        a0 = np.sum(is_not_conf_or_dead*link_weight)

        # The above values aren't the cumulative values, so I handle that below

        if(a1<0):
            print("a1 less than zero")
            #break
            #print(n_per_location[susc_loc][:,2])
        if(a2<0):
            print("a2 less than zero")
            print(n_exp)
        if(a3<0):
            print("a3 less than zero")
            print(n_inf)
        if(a4<0):
            print("a4 less than zero")
        if(a5<0):
            print("a5 less than zero")
        
        a2 = a2 + a1
        a3 = a3 + a2
        a4 = a4 + a3
        a5 = a5 + a4
        
        a0 = a0 + a5
        
        
        dt = -np.log(uniform(0.0, 1.0)) / a0          # Pick the next time step
        t = t + dt                                    # increment time

        a = uniform(0.0,1.0) * a0                     # Choose which process occurs

        k = 0

        if(a<a1):                                     # In this case, an S-> E
            psum = 0
            k=0
            for i in range(0,n_pop):
                if pop[i][0] == 0:
                    n_infected_in_location = inf_by_loc[pop[i][3]]#n_per_location[ pop[i][3] ][2]
                    psum += transmit_rate * n_infected_in_location + background_infection_rate

                    if (psum>a):
                        k=i                            # Index where loop broke
                        #print(pop[k])
                        break

            pop[k][0]=1                                # Set that person to exposed
            
            n_exp = n_exp + 1                          # Increment number of exposed
            n_sus = n_sus - 1                          # Decrement number of susceptible
            
            n_per_location[pop[k][3]][1] += 1          # Increment exposed in location of person
            n_per_location[pop[k][3]][0] -= 1          # Decrement susceptibles in location of person
            

        elif(a<a2):                                    # Here E-> I
            psum = a1
            k=0
            for i in range(0,n_pop):
                if(pop[i][0]==1):
                    psum += e_to_i_rate

                    if (psum>a):
                        k=i
                        break;

            pop[k][0]=2
            n_inf = n_inf + 1                          # Increment number of infected
            n_exp = n_exp - 1                          # Decrement number of exposed

            inf_by_loc[pop[k][3]] += 1 
            n_per_location[pop[k][3]][2] += 1          # Increment infected in location of person
            n_per_location[pop[k][3]][1] -= 1          # Decrement exposed in location of person


        elif(a<a3):                                    # I -> R || D
            psum = a2
            k=0

            for i in range(0,n_pop):

                if(pop[i][0]==2):
                    psum += recovery_rate

                    if(psum>a):
                        k=i
                        break

            is_confined[k] = True


            if(uniform(0.0,1.0)<death_fraction):        # Testing if person died or recovered
                pop[k][0]=4                             # Oh no! Dead!

                n_dead= n_dead+ 1                       # Increment number of recovered
                n_inf = n_inf - 1                       # Decrement number of infected

                inf_by_loc[pop[k][3]] -= 1
                n_per_location[pop[k][3]][4] += 1       # Increment dead in location of person
                n_per_location[pop[k][3]][2] -= 1       # Decrement infected in location of person
                
                is_dead[k] = True                       # Set the person to being dead
                
            else:
                pop[k][0]=3                             # Yay! Recovered!
                
                n_rec = n_rec + 1                       # Increment number of recovered
                n_inf = n_inf - 1                       # Decrement number of infected

                inf_by_loc[pop[k][3]] -= 1
                n_per_location[pop[k][3]][3] += 1       # Increment recovered in location of person
                n_per_location[pop[k][3]][2] -= 1       # Decrement infected in location of person

        elif(a<a4):                                     # Conduct a random test
            psum = a3
            k=0
            for i in range(0,n_pop):
                if(not is_confined[i] and not is_dead[i]):# If the person isn't confined or dead
                    psum += random_test_rate
                    if(psum>a):
                        k=i
                        break

            being_tested[k]=True

        elif(a<a5):
            psum = a4
            k=0
            for i in range(0,n_pop):

                if being_tested[i] == True:
                    psum += finish_test_rate

                    if(psum>a):
                        k=i
                        break

            being_tested[k]=False    

            if (pop[k][0]==2 and uniform(0.0,1.0)<test_sensitivity): # If the person is infected and the test works
                is_confined[k] = True                                # They are confined

                inf_by_loc[pop[k][3]] -= 1
                
                n_per_location[pop[k][3]][2] -= 1          # Decrement infected in current (NET or HOME) location of person
                pop[k][3] = pop[k][1]                      # Send them home
                n_per_location[pop[k][3]][2] += 1          # Increment infected in HOME location of person

                inf_by_loc[pop[k][3]] += 1

        else:                                              # All else failing, move people around
            psum = a5
            k=0
            for i in range(0,n_pop):

                if (not is_confined[i] and not is_dead[i]):  # If they aren't confined or dead
                    psum += link_weight[i]                 # move them to their other location

                    if(psum>a):
                        k=i
                        break

            home_loc = pop[k][1]
            net_loc  = pop[k][2]
#             print(pop[k])
            if(pop[k][3]==home_loc):                       # If the person is at home
                pop[k][3] = net_loc                        # move them to their network location
                
                if ( pop[k][0] == 2):
                    inf_by_loc[home_loc]-= 1
                    inf_by_loc[net_loc] += 1
                
                for j in range(0,5):
                    if(pop[k][0]==j):
                        n_per_location[home_loc][j] -= 1
                        n_per_location[net_loc][j] += 1
                        
# #                         print("Number in home loc"+str(n_per_location[home_loc]))
# #                         print(net_loc)
# #                         print("Number in net loc" + str(n_per_location[net_loc]))
# #                         print()
#                         break
                
                
            else:
                pop[k][3] = home_loc                       # otherwise move them home
                
                if ( pop[k][0] == 2):
                    inf_by_loc[net_loc] += 1
                    inf_by_loc[home_loc] += 1
                
                for j in range(0,5):
                    if(pop[k][0]==j):
                        n_per_location[net_loc][j] -= 1
                        n_per_location[home_loc][j] += 1
                        break
    
        
#         if(np.sum(n_per_location[:,0]) != n_sus):
#             print(str(np.sum(n_per_location[:,0])) + "error   "+ str(n_sus))
#         if(np.sum(n_per_location[:,1]) != n_exp):
#             print(str(np.sum(n_per_location[:,1])) + "error   "+ str(n_exp))
#         if(np.sum(n_per_location[:,2]) != n_inf):
#             print(str(np.sum(n_per_location[:,2])) + "error   "+ str(n_inf))
#         if(np.sum(n_per_location[:,3]) != n_rec):
#             print(str(np.sum(n_per_location[:,3])) + "error   "+ str(n_rec))
#         if(np.sum(n_per_location[:,4]) != n_dead):
#             print(str(np.sum(n_per_location[:,4])) + "error   "+ str(n_dead))
    counter=counter+1
    #print(counter)
    return output


In [5]:
start = time.time()

to_print_to_file = Gillespie_Run(100,output)

end = time.time()

print("Time elapsed: "+str(end-start))

np.savetxt("testing.txt",to_print_to_file,delimiter=" ")

0.0
1.0000223884509825
2.0000049028786187
3.000001000058009
4.000156477806802
5.000242534803125
6.000033338052472
7.000344835977321
8.000043631614322
9.000038071890126
10.000042611659003
11.000383913911723
12.000919844872815
13.000020851928372
14.000051446895936
15.000610385314802
16.00108632126033
17.000625500082794
18.000255350054562
19.00096921301018
20.00029515621186
21.000184772490186
22.000766089373688
23.00029081725913
24.000240592148646
25.000275472152087
26.0001643801519
27.000653365287448
28.000280439271513
29.000478000163184
30.0013651361676
31.000164154041645
32.000497490802495
33.000131885770195
34.00015825028512
35.00174491940562
36.00035895727516
37.000179731101724
38.00074961905379
39.00057580647094
40.000277623261674
41.00135379926679
42.001795703276905
43.00223988585763
44.002166819408984
45.0001325291492
46.0010496734257
47.0016247165558
48.00069551459104
49.000685954241
50.0013021262971
51.00130953398822
52.00020786129328
53.00213503237052
54.00673336434929
55.00173